<a href="https://colab.research.google.com/github/AmanKh9153/Assignment1_solution/blob/main/Assignment1_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importing Necessary Libraries

In [ ]:
from bs4 import BeautifulSoup
import json
import os
import pandas as pd
import re
import requests
import subprocess
import numpy as np

In [ ]:
!pip install datasketch

     |████████████████████████████████| 67 kB 3.1 MB/s 


In [ ]:
from datasketch import MinHash, MinHashLSH
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

### Data Collection --> Scraping Data for years 2017 & 2020

In [ ]:
## Function to Scrape Data from the NIPS website for the years 2017,2020
def get_title_abstract(base_url,year):
    url_paper_list=base_url+'/paper/'+str(year)
    r = requests.get(url_paper_list)
    soup = BeautifulSoup(r.content,'html.parser')
    uls=soup.find_all('ul')
    data1=uls[1]
    paper_link={}
    for li in data1.find_all("li"):
        a=li.find('a')
        paper_link[a.text]=a['href']    
    paper_abstract={}
    for key,item in paper_link.items():
        r1=requests.get(base_url+item)
        soup1=BeautifulSoup(r1.content,'html.parser')
        paras=soup1.find_all('p')
        abstract=paras[-1]
        paper_abstract[key] = abstract
    return paper_abstract

Next we save the title and abstracts of the scraped data as a dictionary indexed by paper title

In [ ]:
paper_abstract_2020=get_title_abstract('https://papers.nips.cc',2020)

In [ ]:
paper_abstract_2017=get_title_abstract('https://papers.nips.cc',2017)

In [ ]:
## Sample element
paper_abstract_2020['Stochastic Latent Actor-Critic: Deep Reinforcement Learning with a Latent Variable Model']

<p>Deep reinforcement learning (RL) algorithms can use high-capacity deep networks to learn directly from image observations. However, these high-dimensional observation spaces present a number of  challenges in practice, since the policy must now solve two problems: representation learning and task learning. In this work, we tackle these two problems separately, by explicitly learning latent representations that can accelerate reinforcement learning from images. We propose the stochastic latent actor-critic (SLAC) algorithm: a sample-efficient and high-performing RL algorithm for learning policies for complex continuous control tasks directly from high-dimensional image inputs. SLAC provides a novel and principled approach for unifying stochastic sequential models and RL into a single method, by learning a compact latent representation and then performing RL in the model's learned latent space. Our experimental evaluation demonstrates that our method outperforms both model-free and mo

In [ ]:
## Next we save the data to a DataFrame as shown below
paper_abstract_2020_list=[]
paper_abstract_2017_list=[]
for key,item in paper_abstract_2020.items():
    temp=[key,item]
    paper_abstract_2020_list.append(temp)
for key,item in paper_abstract_2017.items():
    temp=[key,item]
    paper_abstract_2017_list.append(temp)

In [ ]:
df_2020=pd.DataFrame(paper_abstract_2020_list,columns=['Title','Abstract'])
df_2020

,Title,Abstract
0,A graph similarity for deep learning,[Graph neural networks (GNNs) have been succes...
1,An Unsupervised Information-Theoretic Perceptu...,[Tractable models of human perception have pro...
2,Self-Supervised MultiModal Versatile Networks,[Videos are a rich source of multi-modal super...
3,"Benchmarking Deep Inverse Models over time, an...",[We consider the task of solving generic inver...
4,Off-Policy Evaluation and Learning for Externa...,[We consider the evaluation and training of a ...
...,...,...
1893,Distributed Distillation for On-Device Learning,[On-device learning promises collaborative tra...
1894,COOT: Cooperative Hierarchical Transformer for...,[Many real-world video-text tasks involve diff...
1895,Passport-aware Normalization for Deep Model Pr...,[Despite tremendous success in many applicatio...
1896,Sampling-Decomposable Generative Adversarial R...,[Recommendation techniques are important appro...


In [ ]:
df_2017=pd.DataFrame(paper_abstract_2017_list,columns=['Title','Abstract'])
df_2017

,Title,Abstract
0,Real Time Image Saliency for Black Box Classif...,[In this work we develop a fast saliency detec...
1,Joint distribution optimal transportation for ...,[This paper deals with the unsupervised domain...
2,Learning A Structured Optimal Bipartite Graph ...,[Co-clustering methods have been widely applie...
3,Learning to Inpaint for Image Compression,[We study the design of deep architectures for...
4,Inverse Filtering for Hidden Markov Models,[This paper considers a number of related inve...
...,...,...
674,Generating steganographic images via adversari...,[Adversarial training has proved to be competi...
675,NeuralFDR: Learning Discovery Thresholds from ...,"[As datasets grow richer, an important challen..."
676,A Scale Free Algorithm for Stochastic Bandits ...,[Existing strategies for finite-armed stochast...
677,Value Prediction Network,[This paper proposes a novel deep reinforcemen...


In [ ]:
## Finally we save the data to csv files, which we have included in the data folder in the submission
df_2020.to_csv('Title_Abstract_2020.csv')
df_2017.to_csv('Title_Abstract_2017.csv')

In [ ]:
paper_abstract_2020.keys()

dict_keys(['A graph similarity for deep learning', 'An Unsupervised Information-Theoretic Perceptual Quality Metric', 'Self-Supervised MultiModal Versatile Networks', 'Benchmarking Deep Inverse Models over time, and the Neural-Adjoint method'])

### Reading the Data and Preprocessing steps

In [ ]:
## Setting the root folder containing data
root_folder='/content/drive/MyDrive/DS503Assignment1_11841160_11841200/Q3'

In [ ]:
## Reading the papers (1987-2016) to a dataframe
## Uncomment this for original dataset
# papers_19872016=pd.read_csv(root_folder+'/archive/papers.csv')
##
## Dataframe containing missing abstracts as well. (Using additional data provided)
papers_19872016=pd.read_csv(root_folder+'/AdditionalDataProvided/Dataset3.csv')
papers_19872016

,Unnamed: 0,id,year,title,event_type,pdf_name,abstract,paper_text
0,0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,An efficient method of self-organizing associa...,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,A single cell theory for the development of se...,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,\nIn modeling studies or memory based on neura...,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,"If data collection is costly, there is much to...",Bayesian Query Construction for Neural\nNetwor...
4,4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Learning of continuous valued functions using ...,"Neural Network Ensembles, Cross\nValidation, a..."
...,...,...,...,...,...,...,...,...
7136,7236,994,1994,Single Transistor Learning Synapses,NaN,994-single-transistor-learning-synapses.pdf,We describe single-transistor silicon synapses...,Single Transistor Learning Synapses\n\nPaul Ha...
7137,7237,996,1994,"Bias, Variance and the Combination of Least Sq...",NaN,996-bias-variance-and-the-combination-of-least...,We consider the effect of combining several le...,"Bias, Variance and the Combination of\nLeast S..."
7138,7238,997,1994,A Real Time Clustering CMOS Neural Engine,NaN,997-a-real-time-clustering-cmos-neural-engine.pdf,We describe an analog VLSI implementation of t...,A Real Time Clustering CMOS\nNeural Engine\nT....
7139,7239,998,1994,Learning direction in global motion: two class...,NaN,998-learning-direction-in-global-motion-two-cl...,Perceptual learning is defined as fast improve...,Learning direction in global motion: two\nclas...


In [ ]:
## Removing rows not containing abstract
papers_19872016=papers_19872016[papers_19872016['abstract']!='Abstract Missing'].reset_index()
papers_19872016

,index,Unnamed: 0,id,year,title,event_type,pdf_name,abstract,paper_text
0,0,0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,An efficient method of self-organizing associa...,767\n\nSELF-ORGANIZATION OF ASSOCIATIVE DATABA...
1,1,1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,A single cell theory for the development of se...,683\n\nA MEAN FIELD THEORY OF LAYER IV OF VISU...
2,2,2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,\nIn modeling studies or memory based on neura...,394\n\nSTORING COVARIANCE BY THE ASSOCIATIVE\n...
3,3,3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,"If data collection is costly, there is much to...",Bayesian Query Construction for Neural\nNetwor...
4,4,4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Learning of continuous valued functions using ...,"Neural Network Ensembles, Cross\nValidation, a..."
...,...,...,...,...,...,...,...,...,...
7136,7136,7236,994,1994,Single Transistor Learning Synapses,NaN,994-single-transistor-learning-synapses.pdf,We describe single-transistor silicon synapses...,Single Transistor Learning Synapses\n\nPaul Ha...
7137,7137,7237,996,1994,"Bias, Variance and the Combination of Least Sq...",NaN,996-bias-variance-and-the-combination-of-least...,We consider the effect of combining several le...,"Bias, Variance and the Combination of\nLeast S..."
7138,7138,7238,997,1994,A Real Time Clustering CMOS Neural Engine,NaN,997-a-real-time-clustering-cmos-neural-engine.pdf,We describe an analog VLSI implementation of t...,A Real Time Clustering CMOS\nNeural Engine\nT....
7139,7139,7239,998,1994,Learning direction in global motion: two class...,NaN,998-learning-direction-in-global-motion-two-cl...,Perceptual learning is defined as fast improve...,Learning direction in global motion: two\nclas...


In [ ]:
## Saving the title concatenated with abstract of each paper to a dictionary indexed by paper title
papers_19872016_dic={}
for i,row in papers_19872016.iterrows():
    abstract=row['abstract']
    abstract=abstract.replace('\n','')
    abstract=abstract.replace('\r','')
    papers_19872016_dic[row['title']]=str(row['title'])+' '+abstract

In [ ]:
#papers_19872016_dic

In [ ]:
## Helper function for preprocessing. It will return an array containing tokens (unigrams) for the title,abstract of each paper
import string
from nltk import word_tokenize
def preprocess(mydic):
    temp={}
    for key,item in mydic.items():
        item = item.lower()
        item = "".join([char for char in item if char not in string.punctuation])
        item = word_tokenize(item)
        temp[key] = item
    return temp

In [ ]:
## Example usecase of preprocess
preprocess({"test":"Hi this is a TEst 932"})

{'test': ['hi', 'this', 'is', 'a', 'test', '932']}

In [ ]:
## Helper function for generating kshingles (word shingles or phrases)
def kshingles(mydic,k=1):
  temp={}
  for key,item in mydic.items():
    t1=[]
    if len(item) <= k or k==1:
      t1=item
    else:
      for i in range(0,len(item)-k+1):
        if i == len(item)-k:
          temp_str=''
          for w in item[i:]:
            temp_str+=w
          t1.append(temp_str)
        else:
          temp_str=''
          for w in item[i:i+k]:
            temp_str+=w
          t1.append(temp_str)
    temp[key]=t1
  return temp

In [ ]:
## Example use case of kshingles
kshingles({'a1':['hi','I','am','XYZ'],'a2':['a1','b1','c1','d1','e1']},k=2)

{'a1': ['hiI', 'Iam', 'amXYZ'], 'a2': ['a1b1', 'b1c1', 'c1d1', 'd1e1']}

In [ ]:
## Helper function for computing minhash, perms specifies the number of permutations
def computeminhash(mydic,perms=128):
    temp={}
    for key,item in mydic.items():
        m1 = MinHash(num_perm=perms)
        for d in set(item):
            m1.update(d.encode('utf8'))
        temp[key]=m1
    return temp

In [ ]:
#preprocess({'Algorithms for Non-negative Matrix Factorization': 'Algorithms for Non-negative Matrix Factorization Non-negative matrix factorization (NMF) has previously been shown to be a useful decomposition for multivariate data. Two different multi- plicative algorithms for NMF are analyzed. They differ only slightly in the multiplicative factor used in the update rules. One algorithm can be shown to minimize the conventional least squares error while the other minimizes the generalized Kullback-Leibler divergence. The monotonic convergence of both algorithms can be proven using an auxiliary func- tion analogous to that used for proving convergence of the Expectation- Maximization algorithm. The algorithms can also be interpreted as diag- onally rescaled gradient descent, where the rescaling factor is optimally chosen to ensure convergence. '})

In [ ]:
## Preprocessing and generating minhash. We have used unigrams for further analysis but we can change k to specify k shingles.
papers_19872016_dic=preprocess(papers_19872016_dic)
papers_19872016_dic_k=kshingles(papers_19872016_dic,k=1)
papers_19872016_dic_minhash=computeminhash(papers_19872016_dic_k)

### Building LSH Data Structure

Here we show a use case to build LSH Data Structure for the papers form 1987-2016. However, we build LSH Data Structures again for each of the following questions following our strategy as specified.

In [ ]:
## The threshold specifies the jaccard similarity. Num_Perm specifies the number of permutations used for getting the signature.
lsh = MinHashLSH(threshold=0.3, num_perm=128)
with lsh.insertion_session() as session:
    for key, minhash in papers_19872016_dic_minhash.items():
          session.insert(key, minhash)
print("results for querying an example using LSH\n",lsh.query(list(papers_19872016_dic_minhash.values())[2]))

results for querying an example using LSH
 ['Storing Covariance by the Associative Long-Term Potentiation and Depression of Synaptic Strengths in the Hippocampus', 'Cell Assemblies in Large Sparse Inhibitory Networks of Biologically Realistic Spiking Neurons', 'STDP enables spiking neurons to detect hidden causes of their inputs', 'Estimating time-varying input signals and ion channel states from a single voltage trace of a neuron', 'Clustered factor analysis of multineuronal spike data', 'Homeostatic plasticity in Bayesian spiking networks as Expectation Maximization with posterior constraints', 'Reducing Spike Train Variability: A Computational Theory Of Spike-Timing Dependent Plasticity']


### Original Papers in 2017

In [ ]:
## We build three LSH data structures with different jaccard similarity levels, following the strategy specified in the doc.
lsh = MinHashLSH(threshold=0.3, num_perm=128)
lsh1 = MinHashLSH(threshold=0.6, num_perm=128)
lsh2 = MinHashLSH(threshold=0.9, num_perm=128)
with lsh.insertion_session() as session:
    for key, minhash in papers_19872016_dic_minhash.items():
          session.insert(key, minhash)
with lsh1.insertion_session() as session:
    for key, minhash in papers_19872016_dic_minhash.items():
          session.insert(key, minhash)
with lsh2.insertion_session() as session:
    for key, minhash in papers_19872016_dic_minhash.items():
          session.insert(key, minhash)

In [ ]:
## Reading the data for papers in 2017 to a dictionary indexed by title of the paper.
papers_2017=pd.read_csv(root_folder+'/Title_Abstract_2017.csv')
papers_2017_dic={}
count=0
for i,row in papers_2017.iterrows():
    abstract=row['Abstract']
    abstract=abstract.replace('<p>','').replace('</p>','').replace('<em>','').replace('</em>','')
    papers_2017_dic[row['Title']]=row['Title']+' '+row['Abstract']

In [ ]:
## Performing the necessary preprocessing steps and computing minhash. We have used unigrams.
papers_2017_dic=preprocess(papers_2017_dic)
papers_2017_dic_k=kshingles(papers_2017_dic,k=1)
papers_2017_dic_minhash=computeminhash(papers_2017_dic_k,128)

In [ ]:
#lsh.query(list(papers_2017_dic_minhash.values())[0])

In [ ]:
## Defining our originality score (Kindly refer to pdf submitted). We give a score based on similar documents retrieved at different jaccard similarity levels.
def get_originality_score(minhash):
    list1=lsh.query(minhash)
    list2=lsh.query(minhash)
    list3=lsh.query(minhash)
    score=len(list1)*1 + len(list2)*5 + len(list3)*10
    return 1/score

In [ ]:
## Storing the scores for different papers in 2017
paper_score_dic={}
for key,item in papers_2017_dic_minhash.items():
    paper_score_dic[key] = get_originality_score(item)

In [ ]:
## The top 10 original papers based on our scoring function are as follows
def print_top_k(mydic,k=10):
  count=0
  print("Paper",":","Score")
  for key,value in mydic.items():
    if count==k:
      break
    print(key,":","{:.4f}".format(value))
    count=count+1
print_top_k(dict(sorted(paper_score_dic.items(), key=lambda item: item[1], reverse=True)))

Paper : Score
Maxing and Ranking with Few Assumptions : 0.0625
Exploring Generalization in Deep Learning : 0.0312
Attention is All you Need : 0.0208
A General Framework for Robust Interactive Learning : 0.0208
ELF: An Extensive, Lightweight and Flexible Research Platform for Real-time Strategy Games : 0.0208
Stochastic Approximation for Canonical Correlation Analysis : 0.0156
EEG-GRAPH: A Factor-Graph-Based Model for Capturing Spatial, Temporal, and Observational Relationships in Electroencephalograms : 0.0125
On the Power of Truncated SVD for General High-rank Matrix Estimation Problems : 0.0104
Spectrally-normalized margin bounds for neural networks : 0.0104
Convolutional Phase Retrieval : 0.0089


### Classic Papers

In this section we find the top classic papers which are relevant in 2020.

In [ ]:
from datasketch import MinHashLSHForest, MinHash
# We use MinHashLSHForest from datasketch to answer a top-k query.
forest = MinHashLSHForest(num_perm=128)
for key,minhash in papers_19872016_dic_minhash.items():
    forest.add(key,minhash)
forest.index()

In [ ]:
## Reading the papers of 2020 to a DataFrame
papers_2020 = pd.read_csv(root_folder+'/Title_Abstract_2020.csv')
papers_2020

,Unnamed: 0,Title,Abstract
0,0,A graph similarity for deep learning,<p>Graph neural networks (GNNs) have been succ...
1,1,An Unsupervised Information-Theoretic Perceptu...,<p>Tractable models of human perception have p...
2,2,Self-Supervised MultiModal Versatile Networks,<p>Videos are a rich source of multi-modal sup...
3,3,"Benchmarking Deep Inverse Models over time, an...",<p>We consider the task of solving generic inv...
4,4,Off-Policy Evaluation and Learning for Externa...,<p>We consider the evaluation and training of ...
...,...,...,...
1893,1893,Distributed Distillation for On-Device Learning,<p>On-device learning promises collaborative t...
1894,1894,COOT: Cooperative Hierarchical Transformer for...,<p>Many real-world video-text tasks involve di...
1895,1895,Passport-aware Normalization for Deep Model Pr...,<p>Despite tremendous success in many applicat...
1896,1896,Sampling-Decomposable Generative Adversarial R...,<p>Recommendation techniques are important app...


In [ ]:
## Saving the title, abstract of papers from 2020 to a dictionary indexed by paper titles.
papers_2020_dic={}
count=0
for i,row in papers_2020.iterrows():
    abstract=row['Abstract']
    abstract=abstract.replace('<p>','').replace('</p>','').replace('<em>','').replace('</em>','')
    papers_2020_dic[row['Title']]=row['Title']+' '+row['Abstract']

In [ ]:
## Preprocessing and creating shingles. We use unigrams for further analysis.
papers_2020_dic=preprocess(papers_2020_dic)
papers_2020_dic_k=kshingles(papers_2020_dic,k=1)
papers_2020_dic_minhash=computeminhash(papers_2020_dic_k,128)

In [ ]:
## Sample result for a query of top 10 candidates for an example paper
result = forest.query(list(papers_2020_dic_minhash.values())[8], 10)
print("Top candidates", result)

Top candidates ['Large Margin Discriminant Dimensionality Reduction in Prediction Space', 'Sparse Multi-Task Reinforcement Learning', 'Zeta Hull Pursuits: Learning Nonconvex Data Hulls', 'Heterogeneous multitask learning with joint sparsity constraints', 'A Deep Architecture for Matching Short Texts', 'Exploring Large Feature Spaces with Hierarchical Multiple Kernel Learning', 'Learning with Invariance via Linear Functionals on Reproducing Kernel Hilbert Space', 'Efficient Optimization for Linear Dynamical Systems with Applications to Clustering and Sparse Coding', 'Efficient Second-Order Online Kernel Learning with Adaptive Embedding', 'Scalable Generalized Linear Bandits: Online Computation and Hashing']


In [ ]:
## Store the results of top k queries for each paper in 2020. The LSHForest is formed using papers from 1987-2016.
## Refer to pdf file for more information on strategy employed.
results=[]
for key,item in papers_2020_dic_minhash.items():
    temp=forest.query(item,10)
    results.append(temp)

In [ ]:
np.array(results).shape

(1898, 10)

In [ ]:
## Storing the years of individual papers
years={}
for index,row in papers_19872016.iterrows():
    years[row['title']]=row['year']
#years

In [ ]:
## Our scoring function which returns two types of rankings.
## The first is the set of oldest k papers crossing the score threshold specified.
## The second is the highest scoring papers/ most relevant papers for the entire time span of 1987-2016.
def return_top_scoring_old(results,years,score_threshold=15):
    frequency={}
    for key in years.keys():
        frequency[key]=0
    for item in results:
        for i in item:
            frequency[i]=frequency[i]+1
    top5oldest=[]
    count=0
    for key,item in years.items():
        if count==5:
            break
        if frequency[key]>score_threshold:
            top5oldest.append((key,item,frequency[key]))
            count+=1
    frequency_sorted=dict(sorted(frequency.items(), key=lambda item: item[1], reverse=True))
    top5 = []
    count=0
    for key,item in frequency_sorted.items():
        if count==5:
            break
        top5.append((key,item))
        count=count+1
    return top5oldest,top5

In [ ]:
## Setting the score threshold.
freq_hit_threshold=15
top5oldest,top5=return_top_scoring_old(results,dict(sorted(years.items(), key=lambda item: item[1])),freq_hit_threshold)

In [ ]:
def print_top_k2(mylist,k=5):
  count=0
  for item in mylist:
    if count==k:
      break
    print(item)
    count=count+1
print("Oldest papers with score greater than the frquency hit threshold are (paper,year,score)\n")
print_top_k2(top5oldest)

Oldest papers with score greater than the frquency hit threshold are (paper,year,score)

('Improving the Performance of Radial Basis Function Networks by Learning Center Locations', 1991, 29)
('Extended Regularization Methods for Nonconvergent Model Selection', 1992, 17)
("`N-Body' Problems in Statistical Learning", 2000, 20)
('Maximum Margin Clustering', 2004, 18)
('Multiple Instance Boosting for Object Detection', 2005, 17)


In [ ]:
print("Papers with highest score/frequency from 1987-2006 are (paper,score)\n")
print_top_k2(top5)

Papers with highest score/frequency from 1987-2006 are (paper,score)

('Structure Regularization for Structured Prediction', 34)
('Recursive Segmentation and Recognition Templates for 2D Parsing', 33)
('Dropout Training as Adaptive Regularization', 33)
('Non-parametric Structured Output Networks', 33)
('A Neural Autoregressive Topic Model', 32)


### Thank You.